In [1]:
import tensorflow as tf

train_dir = "./data/train"
valid_dir = "./data/valid"
test_dir  = "./data/test"

img_size = (224, 224)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir, 
    image_size=img_size, 
    batch_size=batch_size
)
valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
    valid_dir, 
    image_size=img_size, 
    batch_size=batch_size
)
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir, 
    image_size=img_size, 
    batch_size=batch_size
)


Found 70295 files belonging to 38 classes.
Found 17572 files belonging to 38 classes.
Found 33 files belonging to 1 classes.


In [2]:
from tensorflow.keras import layers
# Chuẩn hóa và augmentation
data_augmentation = tf.keras.Sequential([
    layers.Rescaling(1./255),
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

In [3]:
from tensorflow.keras import applications
base_model = applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

model = tf.keras.Sequential([
    data_augmentation,
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),
    layers.Dense(38, activation="softmax")  # 38 lớp bệnh
])

In [4]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ ?                      │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,257,984 (8.61 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 2,257,984 (8.61 MB)

In [6]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

checkpoint = ModelCheckpoint("best_model.keras", save_best_only=True, monitor="val_accuracy", mode="max")
early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

history = model.fit(
    train_ds,
    validation_data=valid_ds,
    epochs=10,
    callbacks=[checkpoint, early_stop]
)


Epoch 1/10
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 1371s 624ms/step - accuracy: 0.8739 - loss: 0.4310 - val_accuracy: 0.9096 - val_loss: 0.2833
Epoch 2/10
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 1377s 627ms/step - accuracy: 0.9186 - loss: 0.2494 - val_accuracy: 0.9227 - val_loss: 0.2338
Epoch 3/10
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 1394s 634ms/step - accuracy: 0.9280 - loss: 0.2182 - val_accuracy: 0.9269 - val_loss: 0.2206
Epoch 4/10
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 1401s 638ms/step - accuracy: 0.9308 - loss: 0.2053 - val_accuracy: 0.9336 - val_loss: 0.1985
Epoch 5/10
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 1392s 634ms/step - accuracy: 0.9332 - loss: 0.1968 - val_accuracy: 0.9321 - val_loss: 0.2050
Epoch 6/10
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 1386s 631ms/step - accuracy: 0.9356 - loss: 0.1917 - val_accuracy: 0.9370 - val_loss: 0.1953
Epoch 7/10
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 1387s 631ms/step - accuracy: 0.9360 - loss: 0.1930 - val_accuracy: 0.9406 - val_loss: 0.1812
Epoch 8/10
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 1385s 630ms/s